<a href="https://colab.research.google.com/github/frankwillard/StatGPT/blob/main/GPT_2_HOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]

## Load In Libraries, Data

In [ ]:
import pandas as pd

import torch
from torch.utils.data import Dataset

from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2ForSequenceClassification, TrainingArguments, Trainer

In [ ]:
player_data = pd.read_csv('590.csv')

In [ ]:
player_data.head()

,Unnamed: 0,Player,Eligible,Position,Hall_of_Fame,MVP,Finals_MVP,NBA_Champ,All_NBA,All_Defensive,...,TS%_advanced,OWS_advanced,DWS_advanced,WS_advanced,WS/48_advanced,OBPM_advanced,DBPM_advanced,BPM_advanced,VORP_advanced,peak_ws_advanced
0,0,Alaa Abdelnaby,1,PowerForward,0,0,0,0,0,0,...,0.532,0.7,4.1,4.8,0.072,-2.9,-0.9,-3.8,-1.5,2.1
1,1,Zaid Abdul-Aziz,1,Center,0,0,0,0,0,0,...,0.479,5.9,11.6,17.5,0.076,0.6,-0.2,0.4,2.7,6.5
2,2,Kareem Abdul-Jabbar,1,Center,1,6,2,6,15,11,...,0.592,178.9,94.5,273.4,0.228,4.1,1.6,5.7,85.7,25.4
3,3,Mahmoud Abdul-Rauf,1,PointGuard,0,0,0,0,0,0,...,0.506,16.7,8.4,25.2,0.077,0.7,-1.5,-0.8,4.5,6.8
4,4,Tariq Abdul-Wahad,1,ShootingGuard,0,0,0,0,0,0,...,0.467,-0.6,4.1,3.5,0.035,-2.6,-0.4,-3.0,-1.2,2.2


In [ ]:
df = player_data[["Player", "PTS_per_game", "TRB_per_game" , "AST_per_game" , "STL_per_game" ,"BLK_per_game", "VORP_advanced"]]

df.rename(columns={'PTS_per_game': 'Points', 'TRB_per_game': 'Rebounds', 'AST_per_game': 'Assists', 'STL_per_game': 'Steals', 'BLK_per_game': 'Blocks', "VORP_advanced" : "vorp"}, inplace=True)


# Convert all columns to integer type
df = df.astype(float, errors='ignore')


cond1 = (df["Points"] < 99) & (df["Rebounds"] < 99) & (df["Assists"] < 99) & (df["Steals"] < 99) & (df["Blocks"] < 99)

cond2 = (df["Points"] > 0) & (df["Rebounds"] > 0) & (df["Assists"] > 0) & (df["Steals"] > 0) & (df["Blocks"] > 0)



df = df[cond1]
df = df[cond2]

df.to_csv('players_stats.csv', index=False)

#player_data = player_data[cond]


<ipython-input-64-29ae3e33e66a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'PTS_per_game': 'Points', 'TRB_per_game': 'Rebounds', 'AST_per_game': 'Assists', 'STL_per_game': 'Steals', 'BLK_per_game': 'Blocks', "VORP_advanced" : "vorp"}, inplace=True)
<ipython-input-64-29ae3e33e66a>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[cond2]


In [ ]:
# all_nba = player_data[["Player", "All_NBA"]]
# all_nba.rename(columns={"All_NBA": 'allnba'}, inplace=True)

# all_nba =  all_nba[all_nba["Player"].isin(df["Player"])]
# all_nba.to_csv('all_nba.csv', index=False)

In [ ]:
# vorp = player_data[["Player", "VORP_advanced"]]
# vorp.rename(columns={"VORP_advanced": 'vorp'}, inplace=True)
# vorp =  vorp[vorp["Player"].isin(df["Player"])]
# vorp.head()
# vorp.to_csv('vorp.csv', index=False)


# vorp['vorp'] = vorp['vorp'].apply(lambda x: 0 if x <= 0 else 1)
# vorp.to_csv('vorp_binary.csv', index=False)

In [ ]:
hof = player_data[["Player", "Hall_of_Fame"]]
hof.rename(columns={"Hall_of_Fame": 'hof'}, inplace=True)
len(hof)

<ipython-input-67-bdb431e677e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hof.rename(columns={"Hall_of_Fame": 'hof'}, inplace=True)


5062

In [ ]:
hof =  hof[hof["Player"].isin(df["Player"])]

In [ ]:
hof.head()

,Player,hof
0,Alaa Abdelnaby,0
1,Zaid Abdul-Aziz,0
2,Kareem Abdul-Jabbar,1
3,Mahmoud Abdul-Rauf,0
4,Tariq Abdul-Wahad,0


In [ ]:
len(hof)

3093

In [ ]:
hof.to_csv('hof_binary.csv', index=False)

In [ ]:
# vorp.to_csv('vorp.csv', index=False)

In [ ]:
# vorp['vorp'] = vorp['vorp'].apply(lambda x: 0 if x <= 0 else 1)
# vorp.to_csv('vorp_binary.csv', index=False)

In [ ]:
# df['vorp'] = vorp['vorp'].apply(lambda x: 0 if x <= 0 else 1)
# df['vorp'] = vorp['vorp'].apply(lambda x: 0 if x <= 0 else 1)

## Define Train/Test Split

In [ ]:
player_stats = pd.read_csv('players_stats.csv')
# vorp = pd.read_csv('vorp_binary.csv')
hof = pd.read_csv('hof_binary.csv')

In [ ]:
# prompt: players_stats and vorp have the same players, randomly split the dataset into train and test sets where player_stats are the input and vorp is the label. Players in both sets should be the same. Set the indexing of the pd df to start from 0

from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(player_stats, hof, test_size=0.25, random_state=42)

train_data.index = range(len(train_data))
train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
test_labels.index = range(len(test_labels))


## Convert Tabular Data to Text Input

In [ ]:
training_inputs = []
for index, row in train_data.iterrows():
  # Will include more in future
  train_input = f"{row['Player']} has Points: {row['Points']}, Rebounds: {row['Rebounds']}, Assists: {row['Assists']}, Steals: {row['Steals']}, Blocks: {row['Blocks']}"
  training_inputs.append(train_input)

training_labels = []
for _,row in train_labels.iterrows():
  # Will include more in future
  train_label = f"{row['hof']}"
  training_labels.append(int(train_label))

In [ ]:
# Initialize empty lists for test inputs and labels
testing_inputs = []
testing_labels = []

# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    test_input = f"{row['Player']} has Points: {row['Points']}, Rebounds: {row['Rebounds']}, Assists: {row['Assists']}, Steals: {row['Steals']}, Blocks: {row['Blocks']}"
    testing_inputs.append(test_input)

# Loop through test_labels for labels
for _, row in test_labels.iterrows():
    # Will include more in the future
    test_label = f"{row['hof']}"
    testing_labels.append(int(test_label))


In [ ]:
len(training_inputs)

2319

# Define Model and Start Training

In [ ]:
model_name = "gpt2-large"  # or other sizes like "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model.cuda()



tokenized_training_data = tokenizer(training_inputs, padding=True)
tokenized_testing_data = tokenizer(testing_inputs, padding=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class Dataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.tokenized_data = tokenized_data
        self.labels = labels

    def __len__(self):
        return len(self.tokenized_data["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}

        item["labels"] = torch.tensor(self.labels[idx])

        return item

In [ ]:
# Define training dataset
training_dataset = Dataset(tokenized_training_data, training_labels)
testing_dataset = Dataset(tokenized_testing_data, testing_labels)

In [ ]:
len(training_dataset)

2319

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Define Trainer parameters
def compute_metrics(p):

    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "recall": recall, "f1": f1}

# Define training arguments
training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    seed=0
)


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset,
    compute_metrics=compute_metrics,
)

# Start fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,No log,0.180059,0.963824,0.000000,0.000000
2,No log,0.148922,0.963824,0.000000,0.000000
3,No log,0.094639,0.966408,0.142857,0.235294
4,No log,0.115412,0.965116,0.035714,0.068966
5,No log,0.087156,0.967700,0.178571,0.285714


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=185, training_loss=0.111119337339659, metrics={'train_runtime': 105.4757, 'train_samples_per_second': 109.931, 'train_steps_per_second': 1.754, 'total_flos': 266285375078400.0, 'train_loss': 0.111119337339659, 'epoch': 5.0})

In [ ]:
# count = 0
# total = 0
# batch_size = 64

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)


# model = model.cuda()
# for i in range(0, len(testing_inputs), batch_size):
#     sentences = testing_inputs[i:i+batch_size]
#     labels = torch.tensor(testing_labels[i:i+batch_size])
#     t = tokenizer(sentences, return_tensors="pt", padding = True)
#     t = {key: val.cuda() for key, val in t.items()}

#     pred = model.forward(**t).logits.argmax(-1).detach().cpu()


#     count += (pred == labels).sum()


# print(count)
# print(len(testing_inputs))

In [ ]:
count = 0
total = 0
batch_size = 64

TP, FP, TN, FN = 0, 0, 0, 0

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model = model.cuda()

for i in range(0, len(testing_inputs), batch_size):
    sentences = testing_inputs[i:i + batch_size]
    labels = torch.tensor(testing_labels[i:i + batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding=True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.forward(**t).logits.argmax(-1).detach().cpu()

    TP += ((pred == 1) & (labels == 1)).sum().item()
    TN += ((pred == 0) & (labels == 0)).sum().item()
    FP += ((pred == 1) & (labels == 0)).sum().item()
    FN += ((pred == 0) & (labels == 1)).sum().item()

# Calculating the metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(TP, TN, FP, FN)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

5 744 2 23
Accuracy: 0.9677002583979328
Precision: 0.7142857142857143
Recall: 0.17857142857142858
F1 Score: 0.2857142857142857
